In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
from torchvision import datasets, transforms, models
import torchvision.transforms as transforms

from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import timm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns



D:\python\machine learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:

# defining a class to read images and preprocess and return label values 
class CustomDataset(Dataset):  #creating class and inherit from dataset class of pytorch lib 
    def __init__(self, Directory, transform=None): #defining inputs and methods 
        self.img_path = Directory     
        self.transform = transform
        self.classes = os.listdir(Directory)
        self.image_paths = []
        self.labels = []
        for label, class_folder in enumerate(self.classes): #iterating over subfolders to save labels and path to a list  
            class_folder_path = os.path.join(Directory, class_folder) 
            if os.path.isdir(class_folder_path):
                for image_name in os.listdir(class_folder_path):
                    image_path = os.path.join(class_folder_path, image_name)
                    self.image_paths.append(image_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


directory = r'path to dataset'
val_directory = r'path to validation data'


#defining preprocessing actions(resize, augment with random flip and brightness change, change data type to tensor and normalizing) 
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #mean and standard deviation from imagenet dataset
])
#denormalizing process for output images
denormalize = transforms.Compose([
    transforms.Normalize(mean=[0., 0., 0.], std=[1/0.229, 1/0.224, 1/0.225]),
    transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1., 1., 1.]),
])

#loading and preprocessing data in 32 batches and 0 parallel pools
dataset = CustomDataset(Directory=directory, transform=preprocess)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)
val_dataset = CustomDataset(Directory=val_directory, transform=preprocess)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)




   

22 3


In [3]:
#loading EfficientNet model version b1 and pretrained weights 
model = models.efficientnet_b1(weights='IMAGENET1K_V1')
#train model on GPU(if was available) 
device = torch.device("cuda" if
                      torch.cuda.is_available() else "cpu")
model.to(device)
print(device)
num_classes = 11
#changing last classifier layer of model neurons to match the number of classes 
model.classifier[1] = (
    nn.Linear(model.classifier[1].in_features, num_classes))
#defining loss and optimizations 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters()
                       , lr=0.001,weight_decay=1e-4)
num_epochs = 10
#saving loss and accuracy
history = {'train_loss': [], 'val_accuracy': []}

#train the model 
for epoch in range(num_epochs):
    model.train()   #set model to training mode
    running_loss = 0.0  #reset the loss to show loss in each iteration
    for images, labels in train_loader:
        optimizer.zero_grad() #reset backpropagation value
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()   #computing backpropagation
        optimizer.step()  #updating weights 
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_loader) #calculating loss of each iteration
    
    history['train_loss'].append(epoch_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")
    #setting model to evaluation mode for validation data
    model.eval()
    
    correct = 0
    total = 0
    with (torch.no_grad()): #turning off gradient computation
        for images, labels in val_loader:
            images, labels = images.to(device),
            labels.to(device)   #computing validation data using GPU
            outputs = model(images)
            _, predicted = torch.max(outputs, 1) #predicting output classes
            total += labels.size(0)  #adding number of images in batches 
            correct += (predicted == labels).sum().item() #calculating correct predictions in integer or float format 

    
    accuracy = 100 * correct / total
    history['val_accuracy'].append(accuracy)
    print(f"Validation Accuracy: {accuracy:.2f}%")
    
    
    torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')
    
#plot loss and validation accuracy    
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Training Loss and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

#saving model weights
save_path = r'path to save model'
torch.save(model.state_dict(), save_path)

NameError: name 'models' is not defined

In [2]:
#showing confusion matrix

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=val_dataset.classes, yticklabels=val_dataset.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


print(classification_report(y_true, y_pred, target_names=val_dataset.classes))


NameError: name 'torch' is not defined

In [1]:
#loading and saving images in corresponding folders in specified directory


test_path = r'test images paths'
result_path = r'saved image path'
images = os.listdir(test_path)


model.eval()


if not os.path.exists(result_path):
    os.makedirs(result_path)


for file in images:
    print(file)
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(test_path, file)
        test_image = Image.open(file_path)
        test_image = test_image.convert('RGB')
        test_image = preprocess(test_image)
        test_image = test_image.unsqueeze(0)
        
        
        with torch.no_grad():
            outputs = model(test_image)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            confidence, predicted_class = torch.max(probabilities, 1)
            confidence_percentage = confidence.item() * 100
            
            
            class_label = predicted_class.item()
            class_name = val_dataset.classes[class_label]
            
            
            class_folder = os.path.join(result_path, class_name)
            
            if not os.path.exists(class_folder):
                os.makedirs(class_folder) 

            
            undetected_folder = os.path.join(result_path, 'undetected')
            if not os.path.exists(undetected_folder):
                os.makedirs(undetected_folder)

            #defining a threshold for predictions 
            if confidence_percentage > 45:
            
                result_image_path = os.path.join(class_folder, f"{file}")
                test_image_denorm = denormalize(test_image.squeeze(0))
                test_image_denorm = torch.clamp(test_image_denorm, 0, 1)
                test_image_pil = transforms.ToPILImage()(test_image_denorm)
                test_image_pil.save(result_image_path)
                
                
            
                print(f"Predicted class name: {class_name}")
                print(f"Confidence: {confidence_percentage:.2f}%\n")
                
            else:
                
                undetected_image_path = os.path.join(undetected_folder, file)
                test_image_denorm = denormalize(test_image.squeeze(0))
                test_image_denorm = torch.clamp(test_image_denorm, 0, 1)
                test_image_pil = transforms.ToPILImage()(test_image_denorm)
                test_image_pil.save(undetected_image_path) 
                
                
            
                
                
                print(f"Couldn't detect the tool for {file}\n")
                



KeyboardInterrupt

